In [1]:
import os



In [2]:
data_dir = os.getcwd()

In [3]:
mirzapur_folder = os.path.join(data_dir,'mirzapur')
output_folder = os.path.join(data_dir,'output')

In [4]:
mirzapur_files = os.listdir(mirzapur_folder)
output_files = os.listdir(output_folder)

In [5]:
output_files

[]

In [6]:
from runTransliteration import convert_file
import pysrt


subtitle_s1 =[] 
for  i in mirzapur_files:
    subs = pysrt.open(os.path.join(mirzapur_folder, i))
    subtitle_s1.append(subs)

In [21]:
converted_text = []

for ep in subtitle_s1 :
    for dialogs in ep:
        for dialog in  dialogs.text.split('\n') : 
            converted_text.append(convert_file(dialog).strip())

In [94]:
converted_text[30:400]

['paakoo vidyarthi.',
 'sabhi tumko pasnd karte hain.',
 'aur, in dinon, klej ka mahaul na',
 'thora kharab ho raha hai.',
 'sar, aap abhi tak mudde pe nahin aae.',
 'mudde pe aa rahe hain,',
 'raste men hain.',
 'mudde pe nahin aa rahe hain.',
 'chunav men khari hogi?',
 'dipoo ji pichhe hat ge hain.',
 'aur munna jo hai, vo kaimpas men',
 'kafi laraee-jhgara kar raha hai.',
 'tum agar khari hoti ho,',
 'to usko chunauti milegi.',
 'aur tum agar jit jati ho,',
 'to klej ki thori safaee ho jaegi.',
 'am',
 'ekadam se soch nahin pa rahe hain, sar.',
 'thora sarapraiz kar diya aap hamen.',
 'sarapraiz nahin hai.',
 'tumhare pas abhi bahut vakt hai.',
 'soch lo. sochke hamko batao.',
 'aur kam aasan nahin hai.',
 'aur agar aasan hota,',
 'to ham tumko kyon bolte?',
 'ji, sar.',
 'sochke batate hain, sar.',
 'yar, hamare to kuchh samajh men',
 'nahin aa raha, guddoo bhaiya.',
 'abe, tum hi kuchh batao be.',
 'are, bhaiya, itna to dikha chuke hain.',
 'ab aap hi kuchh bataie.',
 'achchha, r

In [25]:
len(converted_text)

5678

In [30]:

from tensorflow.keras.preprocessing.text import Tokenizer


In [61]:
OOV_TOKEN = "<OOV>"
VOCAB_SIZE = 10000
MAX_LEN = 15

In [62]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN )
tokenizer.fit_on_texts(converted_text)

In [63]:
sequence = tokenizer.texts_to_sequences(converted_text)

In [64]:
n_gram_sequence = []
for i in sequence :
    if len(i)  > 1 :
        for j in range(1,len(i)):
            n_gram_sequence.append(i[:j+1])

In [65]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences = pad_sequences(n_gram_sequence, maxlen=MAX_LEN )

In [66]:
input_sequences.shape

(20704, 15)

In [67]:
## X and Y

x,y = input_sequences[: , :-1], input_sequences[:, -1]

In [68]:
x.shape

(20704, 14)

In [69]:
y

array([646, 358, 647, ...,   6, 384, 109], dtype=int32)

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [71]:
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.models import Sequential

In [77]:
model = Sequential([
    Embedding(VOCAB_SIZE, 20, input_length = (MAX_LEN-1)),
    LSTM(150,return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dense(VOCAB_SIZE, activation = 'softmax')

])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 14, 20)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 14, 150)           102600    
                                                                 
 dropout_2 (Dropout)         (None, 14, 150)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               100400    
                                                                 
 dense_2 (Dense)             (None, 10000)             1010000   
                                                                 
Total params: 1413000 (5.39 MB)
Trainable params: 1413000 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [82]:
history = model.fit(X_train,y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


518/518 [==============================] - 15s 28ms/step - loss: 3.6449 - accuracy: 0.2434 - val_loss: 9.4088 - val_accuracy: 0.1195
Epoch 2/50
518/518 [==============================] - 15s 28ms/step - loss: 3.6215 - accuracy: 0.2460 - val_loss: 9.4794 - val_accuracy: 0.1215
Epoch 3/50
518/518 [==============================] - 15s 29ms/step - loss: 3.5963 - accuracy: 0.2509 - val_loss: 9.5295 - val_accuracy: 0.1215
Epoch 4/50
518/518 [==============================] - 15s 29ms/step - loss: 3.5811 - accuracy: 0.2524 - val_loss: 9.5818 - val_accuracy: 0.1232
Epoch 5/50
518/518 [==============================] - 15s 28ms/step - loss: 3.5566 - accuracy: 0.2582 - val_loss: 9.6424 - val_accuracy: 0.1241
Epoch 6/50
518/518 [==============================] - 15s 29ms/step - loss: 3.5319 - accuracy: 0.2591 - val_loss: 9.7322 - val_accuracy: 0.1229
Epoch 7/50
518/518 [==============================] - 14s 27ms/step - loss: 3.5103 - accuracy: 0.2662 - val_loss: 9.7982 - val_accuracy: 0.1212
Epo

In [151]:
## Predict
import numpy as np
def predict(sentence, next_count):
    words = ""
    predict_sequence = tokenizer.texts_to_sequences([sentence])
    
    for _ in range(next_count):
        
        padd_predict_sequence = pad_sequences(predict_sequence, maxlen=MAX_LEN-1)
        predictions =np.argmax( model.predict(padd_predict_sequence))
        predict_sequence[0].append(predictions)
        word  = tokenizer.index_word[predictions]
        words += word + " "
    
    print(words.strip())

predict('man', 4)
    

1/1 [==============================] - 0s 11ms/step
vale maikadnald pe aa
